In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from FaceRecognitionModel import FaceRecognition,get_device
# from FaceVerificationModel import FaceVerification,get_device
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ColorJitter,transforms

In [2]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [3]:
folder_path = 'lfw_funneled'

In [4]:
image_folders = os.listdir(folder_path)
data_path = []
for path in image_folders:
    if '.' not in path:
        image_path = os.path.join(folder_path,path)
    if len(os.listdir(image_path)) >= 2:
        data_path.append(image_path)
    # break

In [5]:
len(data_path)

1680

In [6]:
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
FaceRecognition = FaceRecognition().to(get_device())

d:\python\Face_verification_Final\FaceRecognitionModel.py:109: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return x.to(get_device())


In [8]:
FaceRecognition.load_state_dict(torch.load('face_recognition_triplet2.pth'))

C:\Users\Kaif Khan\AppData\Local\Temp\ipykernel_22512\3631116834.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  FaceRecognition.load_state_dict(torch.load('face_recogni

<All keys matched successfully>

In [9]:
FaceRecognition.eval()

FaceRecognition(
  (embedding): CNN(
    (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=30976, out_features=1024, bias=True)
    (linear2): Linear(in_features=1024, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=256, bias=True)
  )
)

In [10]:
torch.cuda.empty_cache()

In [11]:
def calculate_distance(embedding1, embedding2):
    return F.pairwise_distance(embedding1, embedding2)

In [12]:
# data_path = data_path[:1000]

In [13]:
# predictions, labels = [],[]
# prediction_true,prediction_false = 0,0
# total_data = 0
# for path1,path2 in zip(data_path[:len(data_path)-1], data_path[1:]):
#     images1 = os.listdir(path1)
#     images2 = os.listdir(path2)

#     anchor_image = Image.open(os.path.join(path1,images1[0]))
#     positive_image = Image.open(os.path.join(path1,images1[1]))
#     negative_image = Image.open(os.path.join(path2,images2[0]))

#     anchor_embedding = FaceRecognition(transform(anchor_image).to(get_device()))
#     positive_embedding = FaceRecognition(transform(positive_image).to(get_device()))
#     negative_embedding = FaceRecognition(transform(negative_image).to(get_device()))

#     same_dist = calculate_distance(anchor_embedding,positive_embedding)
#     diff_dist = calculate_distance(anchor_embedding,negative_embedding)
#     total_data += 2
#     predictions.append(same_dist)
#     predictions.append(diff_dist)

#     labels.append(1)
#     labels.append(0)

#     if same_dist <= 0.5:
#         prediction_true += 1
#     else:
#         prediction_false += 1

#     if diff_dist > 0.5:
#         prediction_true += 1
#     else:
#         prediction_false += 1
# print(f"accuracy of model is {prediction_true/total_data:.2f}")
    
    

In [14]:
# Open files for writing predictions and labels
best_threshold = 1.0
with open('predictions.txt', 'w') as pred_file, open('labels.txt', 'w') as label_file:
    prediction_true, prediction_false = 0, 0
    total_data = 0

    for path1, path2 in zip(data_path[:len(data_path)-1], data_path[1:]):
        images1 = os.listdir(path1)
        images2 = os.listdir(path2)

        anchor_image = Image.open(os.path.join(path1, images1[0]))
        positive_image = Image.open(os.path.join(path1, images1[1]))
        negative_image = Image.open(os.path.join(path2, images2[0]))

        anchor_embedding = FaceRecognition(transform(anchor_image).unsqueeze(0).to(get_device()))
        positive_embedding = FaceRecognition(transform(positive_image).unsqueeze(0).to(get_device()))
        negative_embedding = FaceRecognition(transform(negative_image).unsqueeze(0).to(get_device()))

        same_dist = calculate_distance(anchor_embedding, positive_embedding)
        diff_dist = calculate_distance(anchor_embedding, negative_embedding)
        total_data += 2

        # Write predictions and labels to files
        pred_file.write(f"{same_dist.item()}\n")
        pred_file.write(f"{diff_dist.item()}\n")
        label_file.write("1\n")
        label_file.write("0\n")

        if same_dist <= best_threshold:
            prediction_true += 1
        else:
            prediction_false += 1

        if diff_dist > best_threshold:
            prediction_true += 1
        else:
            prediction_false += 1

    print(f"Accuracy of the model is {prediction_true/total_data:.2f}")


Accuracy of the model is 0.79


In [15]:
print("Prediction, label, total :- ",prediction_true,prediction_false,total_data)

Prediction, label, total :-  2654 704 3358


In [16]:
total_data == prediction_true+prediction_false

True

In [17]:
print(f"accuracy of model is {prediction_true/total_data:.4f}")

accuracy of model is 0.7904


In [18]:
import numpy as np

# Initialize confusion matrix components
TP, TN, FP, FN = 0, 0, 0, 0

# Read predictions and labels from the files
with open('predictions.txt', 'r') as pred_file, open('labels.txt', 'r') as label_file:
    predictions = [float(line.strip()) for line in pred_file.readlines()]
    labels = [int(line.strip()) for line in label_file.readlines()]

# Iterate over predictions and labels to compute TP, TN, FP, FN
for pred, label in zip(predictions, labels):
    if label == 1 and pred <= best_threshold:
        TP += 1
    elif label == 0 and pred > best_threshold:
        TN += 1
    elif label == 0 and pred <= best_threshold:
        FP += 1
    elif label == 1 and pred > best_threshold:
        FN += 1

# Create confusion matrix
confusion_matrix = np.array([[TP, FP], [FN, TN]])

print("Confusion Matrix:")
print(confusion_matrix)


Confusion Matrix:
[[1220  245]
 [ 459 1434]]


In [19]:
# Confusion Matrix:
# [[ 720  339]
#  [ 959 1340]]

In [20]:
from sklearn.metrics import confusion_matrix

# Read predictions and labels from the files
with open('predictions.txt', 'r') as pred_file, open('labels.txt', 'r') as label_file:
    predictions = [float(line.strip()) for line in pred_file.readlines()]
    labels = [int(line.strip()) for line in label_file.readlines()]

# Convert predictions to binary labels using your threshold
threshold = best_threshold
binary_predictions = [1 if pred <= threshold else 0 for pred in predictions]

# Compute confusion matrix using sklearn
cm = confusion_matrix(labels, binary_predictions, labels=[1, 0])

print("Confusion Matrix:")
print(f"TP (True Positives): {cm[0, 0]}")
print(f"FP (False Positives): {cm[1, 0]}")
print(f"FN (False Negatives): {cm[0, 1]}")
print(f"TN (True Negatives): {cm[1, 1]}")


Confusion Matrix:
TP (True Positives): 1220
FP (False Positives): 245
FN (False Negatives): 459
TN (True Negatives): 1434


In [21]:
1651+1619+60+28

3358

In [22]:
folder_path = 'val'

In [23]:
imageFolders = os.listdir(folder_path)
dataPath = []
for folder in imageFolders:
    dataPath.append(os.path.join(folder_path,folder))
dataPath[0],len(dataPath)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'val'

In [24]:
transform1 = transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform2 = transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    RandomHorizontalFlip(),
    RandomRotation(10),
    ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2, hue=0.1)
])

In [25]:
training = []
for path1, path2 in zip(dataPath[:len(dataPath)-1], dataPath[1:]):
    
    images1 = os.listdir(path1)
    images2 = os.listdir(path2)
    # length = 0

    # if(len(images1) < len(images2)):
    #     length = len(images1)
    # else:
    #     length = len(images2)

    # if length < 10:
    #     print(length)
    #     length = 10
    # else:
    #     print(length)
    length = 20
    for i,j in zip(images1[:length], images2[:length]):
        anchorImage = Image.open(os.path.join(path1,images1[0]))
        positiveImage = Image.open(os.path.join(path1,i))
        negaiveImage = Image.open(os.path.join(path2,j))

        anchor = transform1(anchorImage)
        positive = transform1(positiveImage)
        negative = transform1(negaiveImage)

        training.append([anchor, positive, negative])
len(training)


1180

In [26]:
for path1, path2 in zip(dataPath[:len(dataPath)-1], dataPath[1:]):
    
    images1 = os.listdir(path1)
    images2 = os.listdir(path2)
    # length = 0

    # if(len(images1) < len(images2)):
    #     length = len(images1)
    # else:
    #     length = len(images2)

    # if length < 10:
    #     print(length)
    #     length = 10
    # else:
    #     print(length)
    length = 10
    for i,j in zip(images1[:length], images2[:length]):
        anchorImage = Image.open(os.path.join(path1,images1[0]))
        positiveImage = Image.open(os.path.join(path1,i))
        negaiveImage = Image.open(os.path.join(path2,j))

        anchor = transform2(anchorImage)
        positive = transform2(positiveImage)
        negative = transform2(negaiveImage)

        training.append([anchor, positive, negative])

In [27]:
len(training)

1770

In [28]:
training[0][0].shape

torch.Size([3, 200, 200])

In [29]:
best_threshold

1.0

In [30]:
best_threshold = 0.5

In [31]:
with open('predictions.txt', 'w') as pred_file, open('labels.txt', 'w') as label_file:
    prediction_true, prediction_false = 0, 0
    total_data = 0
    for data in training:
        anchor_embedding = FaceRecognition(data[0].unsqueeze(0).to(get_device()))
        positive_embedding = FaceRecognition(data[1].unsqueeze(0).to(get_device()))
        negative_embedding = FaceRecognition(data[2].unsqueeze(0).to(get_device()))

        same_dist = calculate_distance(anchor_embedding, positive_embedding)
        diff_dist = calculate_distance(anchor_embedding, negative_embedding)
        total_data += 2

        # Write predictions and labels to files
        pred_file.write(f"{same_dist.item()}\n")
        pred_file.write(f"{diff_dist.item()}\n")
        label_file.write("1\n")
        label_file.write("0\n")

        if same_dist <= best_threshold:
            prediction_true += 1
        else:
            prediction_false += 1

        if diff_dist > best_threshold:
            prediction_true += 1
        else:
            prediction_false += 1

    print(f"Accuracy of the model is {prediction_true/total_data:.2f}")

Accuracy of the model is 0.60


In [32]:
# Confusion Matrix:
# TP (True Positives): 1220
# FP (False Positives): 245
# FN (False Negatives): 459
# TN (True Negatives): 1434

1220+1434+245+459

3358

In [33]:
from sklearn.metrics import confusion_matrix

# Read predictions and labels from the files
with open('predictions.txt', 'r') as pred_file, open('labels.txt', 'r') as label_file:
    predictions = [float(line.strip()) for line in pred_file.readlines()]
    labels = [int(line.strip()) for line in label_file.readlines()]

# Convert predictions to binary labels using your threshold
threshold = best_threshold
binary_predictions = [1 if pred <= threshold else 0 for pred in predictions]

# Compute confusion matrix using sklearn
cm = confusion_matrix(labels, binary_predictions, labels=[1, 0])

print("Confusion Matrix:")
print(f"TP (True Positives): {cm[0, 0]}")
print(f"FP (False Positives): {cm[1, 0]}")
print(f"FN (False Negatives): {cm[0, 1]}")
print(f"TN (True Negatives): {cm[1, 1]}")


Confusion Matrix:
TP (True Positives): 687
FP (False Positives): 347
FN (False Negatives): 1083
TN (True Negatives): 1423
